In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/소나기-황순원.pdf")
document = loader.load()
document[0].page_content[:200]

'- 1 -소나기\n황순원\n소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀 (曾孫女 )딸이라는 걸 알 수 있었다 . \n소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다 . 서울서는 이런 개울물을 보지 \n못하기나 한 듯이.\n벌써 며칠째 소녀는 , 학교에서 돌아오는 길에 물장난이었다 . 그런데 , 어제까지 개울 기슭에\n서 하더니 , 오늘은 징검다리 한가운'

In [6]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(document)

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)
retriever = docsearch.as_retriever()

In [10]:
from langchain import hub

rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [15]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


In [16]:
from langchain.schema.runnable import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

In [17]:
rag_chain.invoke("소나기는 누구의 작품인가요?")

AIMessage(content='소나기는 황순원의 작품입니다.', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 3250, 'total_tokens': 3259}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c45f7d0b45', 'finish_reason': 'stop', 'logprobs': None}, id='run-7c6ea97a-1212-4b38-95ef-59b066a77789-0')

In [20]:
rag_chain.invoke("주인공에 대해 설명해주세요.")

AIMessage(content='주인공은 소년입니다. 그는 개울가에서 소녀를 처음 만나고, 소녀가 이사 가기 전까지 여러 감정을 겪으며 소녀와의 추억을 쌓습니다. 소녀가 이사 간다는 소식에 안타까워하며, 소녀에게 호두를 주기 위해 몰래 호두를 따기도 합니다.', response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 3405, 'total_tokens': 3485}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None}, id='run-b3778e76-97f1-41a2-b3b4-6d635d6753d2-0')

In [21]:
rag_chain.invoke("마지막 장면은 어떻게 끝나나요?")

AIMessage(content='마지막 장면은 소년이 소녀가 이사 간다는 소식을 듣고, 소녀에게 호두를 전해주기 위해 덕쇠 할아버지네 호두밭에서 호두를 따는 장면으로 끝납니다.', response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 3258, 'total_tokens': 3315}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None}, id='run-e89ef63e-99ff-486a-a3fd-1f334396ac64-0')

In [22]:
rag_chain.invoke("마지막 소년의 부모님들의 대화의 의미는 무엇인가요?")

AIMessage(content='마지막 소년의 부모님들의 대화는 윤 초시 댁의 몰락과 소녀의 죽음을 다루고 있습니다. 윤 초시 댁은 재산을 모두 잃고, 소녀는 병으로 죽었으며, 이는 가문의 대가 끊기는 것을 의미합니다. 소녀가 죽기 전에 자신의 옷을 입혀서 묻어 달라고 한 말이 인상적입니다.', response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 3413, 'total_tokens': 3506}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None}, id='run-08d8d07c-984f-4a38-8334-839ed403f9a4-0')

In [23]:
rag_chain.invoke("왜 소녀가 자신의 옷을 묻어 달라고 했다고 생각하나요?")

AIMessage(content='소녀는 소나기를 맞은 후 분홍 스웨터에 검붉은 진흙물이 들어 잘 지워지지 않자, 그 물이 소년의 등에서 옮은 것이라고 생각했습니다. 소녀는 그 물이 소년과의 추억을 상기시키기 때문에 자신의 옷을 묻어 달라고 한 것 같습니다.', response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 3971, 'total_tokens': 4050}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None}, id='run-a78287e0-ea74-4bae-a2f9-6138ac9509e7-0')

In [24]:
rag_chain.invoke("소년과 소녀는 서로 사랑했다고 생각하나요?")

AIMessage(content='소년과 소녀는 서로 사랑했다고 명확히 말하기는 어렵습니다. 소년은 소녀가 이사 간다는 소식에 허전함을 느끼고, 소녀에게 호두를 주고 싶어하는 등 호감을 보입니다. 그러나 그들의 감정이 사랑인지 단순한 호감인지는 명확하지 않습니다.', response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 3876, 'total_tokens': 3951}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None}, id='run-68ab64c4-0b25-4b24-a1f0-4191f1c8ea55-0')